In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)
import moviepy.editor as mpy

In [3]:
from REINFORCE_helper import RunningVariance
from time import time
from REINFORCE_helper import BaseAgent
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
import keras.backend as K
import numpy as np

Using TensorFlow backend.


In [4]:
class ReinforceAgent(BaseAgent):
    # def __init__(self):
    def get_policy_model(self, lr=0.001, hidden_layer_neurons = 128, input_shape=[4], output_shape=2):
        ## Defino métrica - loss sin el retorno multiplicando
        def loss_metric(y_true, y_pred):
            y_true_norm = K.sign(y_true)
            return K.categorical_crossentropy(y_true_norm, y_pred)
        model = Sequential()
        model.add(Dense(hidden_layer_neurons, input_shape=input_shape, activation='relu'))
        model.add(Dense(output_shape, activation='softmax'))
        ## Por que la categorical_crossentropy funciona ok?
        model.compile(Adam(lr), loss=['categorical_crossentropy'], metrics=[loss_metric])
        return model
    
    def get_action(self, eval=False):
        p = self.model.predict([self.observation.reshape(1, self.nS)])
        if eval is False:
            action = np.random.choice(self.nA, p=p[0]) #np.nan_to_num(p[0])
        else:
            action = np.argmax(p[0])
        action_one_hot = np.zeros(self.nA)
        action_one_hot[action] = 1
        return action, action_one_hot, p
    
    def get_entropy(self, preds, epsilon=1e-12):
        entropy = np.mean(-np.sum(np.log(preds+epsilon)*preds, axis=1)/np.log(self.nA))
        return entropy
    
    def get_critic_model(self, lr=0.001, hidden_layer_neurons = 128, input_shape=[4], output_shape=1):
        model = Sequential()
        model.add(Dense(hidden_layer_neurons, input_shape=input_shape, activation='tanh'))
        model.add(Dense(output_shape, activation='linear'))
        model.compile(Adam(lr), loss=['mse'])
        return model
    
    def get_discounted_rewards(self, r):
        # Por si es una lista
        r = np.array(r, dtype=float)
        """Take 1D float array of rewards and compute discounted reward """
        discounted_r = np.zeros_like(r)
        running_add = 0
        for t in reversed(range(0, r.size)):
            running_add = running_add * self.gamma + r[t]
            discounted_r[t] = running_add
        return discounted_r 

![actor_critic_alg.png](actor_critic_alg.png)

![adv_n_steps.png](adv_n_steps.png)

In [5]:
def get_advantages(values, rewards, gamma=0.999, lmbda=0.95):
    #GAE
    returns = []
    gae = 0
    for i in reversed(range(len(rewards))):
        delta = rewards[i] + gamma * values[i + 1] - values[i]
        gae = delta + gamma * lmbda * gae
        returns.insert(0, gae + values[i])

    adv = np.array(returns) - values[:-1]
    return np.array(returns), adv

In [6]:
# print(compute_n_step_targets(rewards, values, gamma=reinforce_agent.gamma,  n_steps = 1)[-10:].reshape(-1))
# print((rewards.reshape(-1,1) + reinforce_agent.gamma*values[1:] - values[:-1])[-10:].reshape(-1))

In [7]:
# print(compute_n_step_targets(rewards, values, gamma=reinforce_agent.gamma,  n_steps = len(rewards))[:10].reshape(-1))
# print((disc_sum_rews.reshape(-1, 1) - values[:-1])[:10].reshape(-1))

In [8]:
reinforce_agent = ReinforceAgent('LunarLander-v2', n_experience_episodes=1, EPISODES=2000, epochs=1, 
                                 lr=0.01, algorithm='ACTOR_CRITIC_GAE', gif_to_board=True, batch_size=128)

initial_time = time()
running_variance = RunningVariance()
critic_model = reinforce_agent.get_critic_model(lr=0.001, 
                                           hidden_layer_neurons=128,
                                           input_shape=[reinforce_agent.nS],
                                           output_shape=1)


while reinforce_agent.episode < reinforce_agent.EPISODES:
    obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs, time_steps = reinforce_agent.get_experience_episodes(return_ts=True)
    
    history_critic = critic_model.fit(obs, disc_sum_rews, verbose=0, epochs=1, batch_size=reinforce_agent.batch_size)
    
    all_obs = np.vstack([obs, [last_obs]])
    values = critic_model.predict(all_obs)
            
    _, advantage = get_advantages(values, rewards, gamma=reinforce_agent.gamma, lmbda=0.6)

    for ad in advantage:
        running_variance.add(ad)

    pseudolabels = actions*advantage
    
    history_loss = reinforce_agent.model.fit(obs, pseudolabels, verbose=0, epochs=reinforce_agent.epochs, batch_size=reinforce_agent.batch_size)
    
    reinforce_agent.log_data(reinforce_agent.episode, 
                      history_loss.history['loss'][0], 
                      np.mean(ep_len), 
                      reinforce_agent.get_entropy(preds), 
                      running_variance.get_variance(), 
                      history_loss.history['loss_metric'][0], 
                      time() - initial_time, np.mean(ep_returns[-1]), 
                      history_critic.history['loss'][0])
    
reinforce_agent.writer.close()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
correr en linea de comando: tensorboard --logdir logs/
Episode: 50

t:   0%|          | 0/121 [00:00<?, ?it/s, now=None]


Model on episode 51 improved from -inf to -733.7070167916418. Saved!
MoviePy - Building file /tmp/tmpma8idmvp.gif with imageio.


Episode: 102

t:   0%|          | 0/97 [00:00<?, ?it/s, now=None]

...

Episode: 778
Model on episode 779 improved from 15.832457775165178 to 112.00550987142347. Saved!


t:   3%|▎         | 14/497 [00:00<00:03, 133.37it/s, now=None]

MoviePy - Building file /tmp/tmpvctbhk6o.gif with imageio.


Episode: 1161

KeyboardInterrupt: 